In [1]:
! pip install finance-datareader

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import itertools
import random

import FinanceDataReader as fdr
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [3]:
random.seed(1234)
np.random.seed(1234)
os.environ["PYTHONHASHSEED"] = str(1234)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = '/content/drive/MyDrive/주식 종가 예측/open/'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [6]:
start_date = '20210104'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 47
HOW MANY "Business_days" : (235, 1)


# Sample

In [7]:
sample_code = stock_list.loc[0, '종목코드']
sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample.fillna(method='ffill', inplace=True)

In [ ]:
! pip install pmdarima

In [11]:
from pmdarima.arima import auto_arima

In [24]:
model = auto_arima(train['Close'].values.astype('float64'), error_action='ignore', suppress_warnings=True)

In [25]:
model_fit = model.fit(train['Close'].values.astype('float64'))

In [26]:
test_public = sample[-20:-15]['Close'].reset_index(drop = True)
test_public = np.array(test_public)

In [27]:
pred_public = model_fit.predict(n_periods = 5)

In [28]:
np.mean(np.abs(test_public - pred_public) / test_public)

0.010151803334136858

# Final

In [7]:
! pip install pmdarima

In [8]:
nmae_df = pd.DataFrame(columns = ['code', 'nmae'])

In [9]:
submission_name = 'sample_submission.csv'
submission = pd.read_csv(os.path.join(path,submission_name))

In [ ]:
from pmdarima.arima import auto_arima
import warnings
import time

warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
  print(f'\n code: {code} \n')

  # Load data
  stock_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_data.fillna(method = 'ffill', inplace = True)

  # Split the data into train / validation / test sets
  train_public = stock_data[:-20] # ~ 10/29 : predict from 11/1 to 11/5
  train_private = stock_data # ~ 11/26 : predict from 11/29 to 12/3

  test_public = stock_data[-20:-15]['Close'].reset_index(drop = True) # from 11/1 to 11/5
  test_public = np.array(test_public)

  # Fit the model with selected p, d, q values
  public_model = auto_arima(train_public['Close'].values.astype('float64'), 
                            start_p = 1, start_q = 1,
                            start_P = 0, start_Q = 0,
                            m = 73,
                            error_action = 'ignore', suppress_warnings = True)
  public_model_fitted = public_model.fit(train_public['Close'].values.astype('float64'))
  public_forecast_result = public_model_fitted.predict(n_periods = 5)

  pred_public = public_forecast_result

  # Check nmae values
  public_nmae = np.mean(np.abs(test_public - pred_public) / test_public)
  nmae_df = nmae_df.append({'code' : code, 'nmae' : public_nmae}, ignore_index = True)
    
  print('public predicted values: ', pred_public, '\n')
  print('nmae: ', public_nmae, '\n')

  # Add predicted valued to submission file
  for i in range (0, 5):
    submission.loc[i, code] = pred_public[i]

  private_model = auto_arima(train_private['Close'].values.astype('float64'),
                             start_p = 1, start_q = 1,
                             start_P = 0, start_Q = 0,
                             m = 73,
                             error_action = 'ignore', suppress_warnings = True)
  private_model_fitted = private_model.fit(train_private['Close'].values.astype('float64'))
  private_forecast_result = private_model_fitted.predict(n_periods = 5)

  pred_private = private_forecast_result

  print('private predicted values: ', pred_private, '\n')

  # Add predicted valued to submission file
  for i in range (0, 5):
    submission.loc[i+5, code] = pred_private[i]

  time.sleep(1)

  0%|          | 0/370 [00:00<?, ?it/s]


 code: 005930 

public predicted values:  [69800. 69800. 69800. 69800. 69800.] 

nmae:  0.010151803334136858 

private predicted values:  [72300. 72300. 72300. 72300. 72300.] 



  0%|          | 1/370 [00:30<3:08:46, 30.70s/it]


 code: 000660 

public predicted values:  [103000. 103000. 103000. 103000. 103000.] 

nmae:  0.03282121234170771 

private predicted values:  [115500. 115500. 115500. 115500. 115500.] 



  1%|          | 2/370 [01:13<3:51:58, 37.82s/it]

...


 code: 023530 

public predicted values:  [105000. 105000. 105000. 105000. 105000.] 

nmae:  0.020492888534786792 

private predicted values:  [90800. 90800. 90800. 90800. 90800.] 



 27%|██▋       | 100/370 [3:12:01<2:49:53, 37.75s/it]


 code: 039490 

public predicted values:  [105276.12184137 104993.83443355 104651.02585556 104352.8199985
 104136.13825811] 

nmae:  0.013428576229728179 

private predicted values:  [102987.4389712  102993.83419898 102783.99565677 102492.79227105
 102396.00511925] 



 27%|██▋       | 101/370 [3:28:32<24:11:50, 323.83s/it]


 code: 047040 



In [ ]:
# nmae 평균
nmae_result = np.array(nmae_df['nmae'])

In [ ]:
np.mean(nmae_result)

0.017420893394797365

In [13]:
pred_public

69800.0